In [159]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torchvision.datasets.folder import default_loader
import os

# Directory containing images
image_dir = 'images/'
image_files = os.listdir(image_dir)

# Preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load pre-trained model
model = resnet50(pretrained=True)
model.eval()  # Set model to evaluation mode

# Feature extraction function
def extract_features(image_path):
    image = default_loader(image_path)  # Load image as PIL.Image
    input_tensor = preprocess(image)  # Preprocess image
    input_batch = input_tensor.unsqueeze(0)  # Create a mini-batch as expected by the model

    with torch.no_grad():
        features = model(input_batch)
    
    return features

# Extract features from all images
features_list = []
for filename in image_files:
    if filename.endswith(".jpeg"):  # Filter for jpeg images
        image_path = os.path.join(image_dir, filename)
        features = extract_features(image_path)
        features_list.append(features)

/Users/khanhdam/.local/share/virtualenvs/LLMs-2-OK3sre9M/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/khanhdam/.local/share/virtualenvs/LLMs-2-OK3sre9M/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [160]:
import numpy as np

In [161]:
np.array(features_list[1]).shape

(1, 1000)

In [162]:
# import csv
# import numpy as np

# # Path to the output CSV file
# output_csv_path = 'image_features/features.csv'

# # Open a file with writing permission
# with open(output_csv_path, 'w', newline='') as csvfile:
#     # Create a csv writer
#     csvwriter = csv.writer(csvfile)

#     # Write a header row (optional, if you want column headers)
#     headers = [f'Feature_{i}' for i in range(features_list[0].numel())]
#     csvwriter.writerow(headers)

#     # Write the features to the csv file
#     for features in features_list:
#         # Convert tensor to a list (or you can use .numpy().tolist() if you prefer NumPy)
#         features_data = features.squeeze().tolist()  # Ensure it's a 1D list
#         csvwriter.writerow(features_data)

In [164]:
descriptions = []
descriptions.append("The cat is on the sofa")
descriptions.append("The cat is not looking at the camera")
descriptions.append("The dos is playing")
descriptions.append("The dog is lying on the floor")

In [165]:
import csv
# Path to the CSV file where you want to save the data
csv_file_path = 'image_features/features_with_descriptions.csv'

# Write features and descriptions to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    count = 1
    # Writing headers
    csvwriter.writerow(['ID', 'Description', 'Feature'])
    for description, features in zip(descriptions, features_list):
        id = count
        # Convert features tensor to a list for CSV writing
        features_list = features.squeeze().tolist()  # Squeeze to remove batch dimension and convert to list
        csvwriter.writerow([id, description, features_list])
        count = count + 1

In [166]:
import pandas as pd
df = pd.read_csv('image_features/features_with_descriptions.csv')
df.shape

(4, 3)

In [167]:
df

,ID,Description,Feature
0,1,The cat is on the sofa,"[0.2631637454032898, -1.2990944385528564, -1.6..."
1,2,The cat is not looking at the camera,"[-0.8799721598625183, 0.12256206572055817, -3...."
2,3,The dos is playing,"[0.15154120326042175, -0.06807504594326019, -2..."
3,4,The dog is lying on the floor,"[-0.10581132769584656, -1.1986559629440308, -1..."


In [168]:
import csv

csv_file_path = 'image_features/features_with_descriptions.csv'

data = []
with open(csv_file_path, mode='r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        id = row['ID']
        description = row['Description']
        # Convert string representation of list back to a list of floats
        feature = [float(x) for x in row['Feature'].strip('[]').split(',')]
        data.append({'id': id, 'description': description, 'feature': feature})

In [169]:
from qdrant_client import models, QdrantClient
qdrant = QdrantClient(host='localhost', port=6333)

In [176]:
qdrant.recreate_collection(
    collection_name="my_images",
    vectors_config=models.VectorParams(
        size=1000,
        distance=models.Distance.COSINE,
    ),
)

True

In [177]:
data

[{'id': '1',
  'description': 'The cat is on the sofa',
  'feature': [0.2631637454032898,
   -1.2990944385528564,
   -1.6400659084320068,
   0.6857174634933472,
   -1.6096782684326172,
   1.2166526317596436,
   -2.312701463699341,
   -0.6139358282089233,
   -1.1444718837738037,
   -2.987438440322876,
   1.5173687934875488,
   -1.5400828123092651,
   -1.9780714511871338,
   -2.3931021690368652,
   -3.5895450115203857,
   -2.5405354499816895,
   0.26680827140808105,
   -1.3058302402496338,
   -2.580500364303589,
   -2.8725874423980713,
   -3.8474254608154297,
   -1.4030325412750244,
   -5.390924453735352,
   -1.3204879760742188,
   -4.946127414703369,
   0.002371758222579956,
   -0.6447705626487732,
   -2.601376533508301,
   -0.9838295578956604,
   1.2767075300216675,
   -2.0865678787231445,
   -2.1531472206115723,
   0.0073188841342926025,
   -0.7727858424186707,
   -0.07349491119384766,
   -1.539932370185852,
   -0.18488915264606476,
   -1.9956433773040771,
   2.234128952026367,
   -2.

In [178]:
qdrant.upload_points(
    collection_name="my_images",
    points=[
        models.PointStruct(
            id=int(doc["id"]), 
            vector=doc['feature'], 
            payload={
                "description": doc['description']
            }
        )
        for doc in data
    ],
)

In [183]:
test = 'test_images/test2.jpeg'
feature = []
test_feature = extract_features(test).squeeze().tolist()
test_feature = [float(x) for x in test_feature]

In [184]:
test_feature

[0.6397356390953064,
 0.32934609055519104,
 -2.6631290912628174,
 -2.815931797027588,
 -1.9632611274719238,
 -0.9479631185531616,
 -5.662021160125732,
 -1.6118261814117432,
 -1.5514521598815918,
 -2.2050836086273193,
 -0.7831017971038818,
 -2.055140256881714,
 -1.500549077987671,
 -3.868741035461426,
 -0.3137798309326172,
 -1.4530218839645386,
 -1.7150909900665283,
 -1.8290657997131348,
 -2.095214366912842,
 -2.717024087905884,
 -1.3188623189926147,
 -0.39471057057380676,
 -3.6178600788116455,
 -1.5946707725524902,
 -2.194194793701172,
 0.9055305123329163,
 0.19603025913238525,
 -0.058647096157073975,
 0.30464962124824524,
 -0.6613630056381226,
 -1.4290595054626465,
 0.22222882509231567,
 1.784441351890564,
 -1.9342632293701172,
 0.2506401836872101,
 -0.8536369204521179,
 -0.34938758611679077,
 -1.6997714042663574,
 -0.25908446311950684,
 -0.4622611999511719,
 -0.7964177131652832,
 -0.14877185225486755,
 0.4546457529067993,
 -1.3501479625701904,
 -1.4847066402435303,
 -0.36137324571609

In [185]:
hits = qdrant.search(
    collection_name="my_images",
    query_vector=test_feature,
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'description': 'The dog is lying on the floor'} score: 0.6368863
{'description': 'The cat is on the sofa'} score: 0.46313396
{'description': 'The dos is playing'} score: 0.29161456
